In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import sys

In [30]:
dbpedia_endpoint = "https://dbpedia.org/sparql"
sparql = SPARQLWrapper(dbpedia_endpoint, agent='sparqlwrapper 1.8.6.dev0 (rdflib.github.io/sparqlwrapper)')

In [5]:
class_qry1 = """PREFIX wikiID: <http://www.wikidata.org/entity/>
select distinct ?ent ?type where { 
?ent owl:sameAs wikiID:"""
#Q76 to come here in between
class_qry2 = """ . ?ent rdf:type ?type 
FILTER( regex( ?type, "http://dbpedia.org/ontology/"))
}"""

In [ ]:
for i in range(4):
    inp_file = '../data_files/simpques_wiki_smart/outdata'+str(i+1)+'.json'
    out_file = '../data_files/simpques_wiki_smart/output_sq_res'+str(i+1)+'.json'
    df_sq = pd.read_json(inp_file)
    #print(len(df_sq))
    lst_id, lst_ques, lst_datatype, lst_res = [], [], [], []
    for i in range(len(df_sq)):
        lst_id.append(df_sq['simple_questions_wikidata'][i]['id'])
        lst_ques.append(df_sq['simple_questions_wikidata'][i]['question'])
        wiki_obj = df_sq['simple_questions_wikidata'][i]['object']
        qry = class_qry1 + wiki_obj + class_qry2
        #print(qry)
        dbpedia_catg, dbpedia_type = fetch_ent_typ(qry)  #should return a category and a list 
        #print(dbpedia_catg, dbpedia_type)
        lst_datatype.append(dbpedia_catg)
        lst_res.append(dbpedia_type)
    pdf_data_v = {'id': lst_id, 'question': lst_ques, 'category': lst_datatype, 'type': lst_res}
    new_df = pd.DataFrame(pdf_data_v, columns = ['id','question','category','type'])
    new_df.to_json(out_file, orient='records', indent=len(new_df.columns))

In [43]:
#function to fetch results of search term
def fetch_ent_typ(qry):
       dbpedia_catg = ''
       lst_dbpd_typs = []
       try:
          sparql.setQuery(qry)
          sparql.setReturnFormat(JSON)
          results = sparql.query().convert()    #results['boolean']
          #print(results)
          if not len(results['results']['bindings']) == 0:
             for item in results['results']['bindings']:                   
                try:
                   dbpedia_catg = item['type']['type']
                   #print(dbpedia_catg)
                   lst_dbpd_typs.append(item['type']['value'].split('/')[-1])
                except Exception as ex:
                   print(ex)
                   pass
       except Exception as ex:
          print('exception came: ',ex)
          pass
       if dbpedia_catg == 'uri':
          dbpedia_catg = 'resource'
       return dbpedia_catg, lst_dbpd_typs